First, import and initiailize the python earthengine api package

In [1]:
import ee
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


If the initialization succeeded, you can stop here. Congratulations! If not, continue on below...

#### Authenticating to the Earth Engine servers
If the initialization process failed, you will need to authenticate the Jupyter Notebook server so that it can communicate with the Earth Engine servers. You can initiate the authentication process by running the following bash command.

In [8]:
%%bash
earthengine authenticate --quiet

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --authorization-code=PLACE_AUTH_CODE_HERE



Once you have obtained an authorization code from the previous step, paste the code into the following cell and run it.

In [9]:
%%bash
earthengine authenticate --authorization-code=PLACE_AUTH_CODE_HERE


Successfully saved authorization token.


In [2]:
# standard modules
import ee
from pprint import pprint
import datetime
import math
import pickle
ee.Initialize()
import os
import sys
from geetools import ui, cloud_mask, batch

# package modules
from atmcorr.atmospheric import Atmospheric
from atmcorr.timeSeries import timeSeries


## User Inputs

In [3]:
# start and end of time series
START_DATE = '2017-01-01'  # YYYY-MM-DD
STOP_DATE = '2017-12-31'  # YYYY-MM-DD

# define YOUR GEE asset path (check the Code Editor on the Google Earth Engine Platform)
assetPath = 'users/visithuruvixen/'

# Location
studyarea = ee.Geometry.Rectangle(8.212529728804157, 60.11440450337899, 8.025762150679157, 60.02086917002816)
#sitepoint= ee.Geometry.Point(8.1191,60.0676) #fill in the more accuracte coordinates here

# Description of time period and location
assetID = 'year2017'

Other variables that need setting here

In [4]:
# satellite missions, 
MISSIONS = ['Sentinel2']
NO_OF_BANDS = 13

# Location of iLUTs (can keep default if you clone entire git repository to your machine)
DIRPATH = './files/iLUTs/S2A_MSI/Continental/view_zenith_0/'

# setting parameter for atmospheric correction
SRTM = ee.Image('USGS/GMTED2010')  # Make sure that your study area is covered by this elevation dataset
altitude = SRTM.reduceRegion(reducer=ee.Reducer.mean(), geometry=studyarea.centroid()).get('be75').getInfo() # insert correct name for elevation variable from dataset
KM = altitude/1000  # i.e. Py6S uses units of kilometers

In [26]:
target = assetID
# the following creates interpolated lookup tables.
_ = timeSeries(target, studyarea, START_DATE, STOP_DATE, MISSIONS) #You can also create lookup tables for other missions if desired 

Loading interpolated look up tables (.ilut) for Sentinel2..
Success
Getting data from Earth Engine.. 


AttributeError: 'ServerNotFoundError' object has no attribute 'message'

### Getting an Image Collection from the GEE Server

In [5]:
# The Sentinel-2 image collection
S2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(studyarea)\
       .filterDate(START_DATE, STOP_DATE).sort('system:time_start')\
       .map(cloud_mask.sentinel2()) # applies an ESA cloud mask on all images (L1C)
S2List = S2.toList(S2.size()) # must loop through lists

NO_OF_IMAGES = S2.size().getInfo()  # no. of images in the collection

### defining functions for atmospheric correction

In [6]:
def atm_corr_image(imageInfo: dict) -> dict:
    """Retrieves atmospheric params from image.

    imageInfo is a dictionary created from an ee.Image object
    """
    atmParams = {}
    # Python uses seconds, EE uses milliseconds:
    scene_date = datetime.datetime.utcfromtimestamp(imageInfo['system:time_start']/1000)
    dt1 = ee.Date(str(scene_date).rsplit(sep=' ')[0])

    atmParams['doy'] = scene_date.timetuple().tm_yday
    atmParams['solar_z'] = imageInfo['MEAN_SOLAR_ZENITH_ANGLE']
    atmParams['h2o'] = Atmospheric.water(geom, dt1).getInfo()
    atmParams['o3'] = Atmospheric.ozone(geom, dt1).getInfo()
    atmParams['aot'] = Atmospheric.aerosol(geom, dt1).getInfo()
    return atmParams


def get_corr_coef(imageInfo: dict, atmParams: dict) -> list:
    """Gets correction coefficients for each band in the image.

    Uses DIRPATH global variable
    Uses NO_OF_BANDS global variable
    Uses KM global variable
    Returns list of 2-length lists
    """
    corr_coefs = []
    # string list with padding of 2
    bandNos = [str(i).zfill(2) for i in range(1, NO_OF_BANDS + 1)]
    for band in bandNos:
        filepath = DIRPATH + 'S2A_MSI_' + band + '.ilut'
        with open(filepath, 'rb') as ilut_file:
            iluTable = pickle.load(ilut_file)
        a, b = iluTable(atmParams['solar_z'], atmParams['h2o'], atmParams['o3'], atmParams['aot'], KM)
        elliptical_orbit_correction = 0.03275104*math.cos(atmParams['doy']/59.66638337) + 0.96804905
        a *= elliptical_orbit_correction
        b *= elliptical_orbit_correction
        corr_coefs.append([a, b])
    return corr_coefs


def toa_to_rad_multiplier(bandname: str, imageInfo: dict, atmParams: dict) -> float:
    """Returns a multiplier for converting TOA reflectance to radiance

    bandname is a string like 'B1'
    """
    ESUN = imageInfo['SOLAR_IRRADIANCE_'+bandname]
    # solar exoatmospheric spectral irradiance
    solar_angle_correction = math.cos(math.radians(atmParams['solar_z']))
    # Earth-Sun distance (from day of year)
    d = 1 - 0.01672 * math.cos(0.9856 * (atmParams['doy']-4))
    # http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    return multiplier


def atm_corr_band(image, imageInfo: dict, atmParams: dict):
    """Atmospherically correct image

    Converts toa reflectance to radiance.
    Applies correction coefficients to get surface reflectance
    Returns ee.Image object
    """
    oldImage = ee.Image(image).divide(10000)
    newImage = ee.Image()
    cor_coeff_list = get_corr_coef(imageInfo, atmParams)
    bandnames = oldImage.bandNames().getInfo()
    for ii in range(NO_OF_BANDS):
        img2RadMultiplier = toa_to_rad_multiplier(bandnames[ii], imageInfo, atmParams)
        imgRad = oldImage.select(bandnames[ii]).multiply(img2RadMultiplier)
        constImageA = ee.Image.constant(cor_coeff_list[ii][0])
        constImageB = ee.Image.constant(cor_coeff_list[ii][1])
        surRef = imgRad.subtract(constImageA).divide(constImageB)
        newImage = newImage.addBands(surRef)
    # unpack a list of the band indexes:
    return newImage.select(*list(range(NO_OF_BANDS)))

In [ ]:
#date = ee.Date(dateString)
geom = studyarea

S3 = S2List
SrList = ee.List([0]) # Can't init empty list so need a garbage element
export_list = []
coeff_list = []
for i in range(NO_OF_IMAGES):
    iInfo = S3.get(i).getInfo()
    iInfoProps = iInfo['properties']
    atmVars = atm_corr_image(iInfoProps)
    corrCoeffs = get_corr_coef(iInfoProps, atmVars)
    coeff_list.append(corrCoeffs)
    # # set some properties to tack on to export images
    #info = S3.getInfo()['properties']  #called iInfo
    scene_date = datetime.datetime.utcfromtimestamp(iInfoProps['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
    dateString = scene_date.strftime("%Y-%m-%d")
    
    # # Atmospheric constituents
    h2o = Atmospheric.water(geom,ee.Date(dateString)).getInfo()
    o3 = Atmospheric.ozone(geom,ee.Date(dateString)).getInfo()
    aot = Atmospheric.aerosol(geom,ee.Date(dateString)).getInfo()
    
    img = atm_corr_band(ee.Image(S3.get(i)), iInfoProps, atmVars)
    img = img.set({'satellite':'Sentinel 2',
              'fileID':iInfoProps['system:index'],
              'date':dateString,
              'aerosol_optical_thickness':aot,
              'water_vapour':h2o,
              'ozone':o3})
    SrList = SrList.add(img)

SrList = SrList.slice(1) # Need to remove the first element from the list which is garbage
with open('coeff_list.txt', 'w') as f:
    pprint(coeff_list, stream=f)

In [33]:
#SrList.size().getInfo()==S2.size().getInfo()
SrList.size().getInfo()

6

In [10]:
firstImagenotcor = ee.Image(S2List.get(3)).divide(10000)
firstImageatcor = ee.Image(SrList.get(3))
firstImagenotcor

In [11]:
from IPython.display import display, Image

region = geom.buffer(10000).bounds().getInfo()['coordinates']
channels = ['B4','B3','B2']

before = Image(url=firstImagenotcor.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25#,'gamma':1.5
                }))

after = Image(url=firstImageatcor.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25#,'gamma':1.5
                }))

display(before, after)

In [12]:
Map = ui.Map(tabs=('Inspector',))
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [18]:
imageidx=6
firstImagenotcor = ee.Image(S2List.get(imageidx)).divide(10000)
firstImageatcor = ee.Image(SrList.get(imageidx))
CorCol = ee.ImageCollection(SrList)#.map(cloud_mask.sentinel2()) #converting the list of atcor images to an imagecollection
vis = {'bands':['B4', 'B3','B2'], 'min':0, 'max':0.3}
#visS2 = {min: 0.0,max: 0.25,'bands':channels}
#is2=is2.clip(aoi)
from geetools import ui, tools, composite, cloud_mask, indices
bands=['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']
#medoid = composite.medoid(CorCol, bands=bands)
image = S2.mosaic()
img = CorCol.mosaic()

In [19]:
Map.centerObject(firstImagenotcor.clip(geom), zoom=11)
Map.addLayer(firstImagenotcor.clip(geom),vis, 'Uncorrected original, cloud masked')
Map.addLayer(firstImageatcor.clip(geom),vis, 'Atmospherically corrected')
#Map.addLayer(CorCol.first().clip(geom),vis, 'Atmospherically corrected, cloud masked')
#Map.addLayer(medoid.clip(geom), vis, 'Medoid AtCorrected')
Map.addLayer(S2.mosaic().clip(geom), {'bands':['B4', 'B3','B2'], 'min':0, 'max':5000}, 'Mosaic Not Corrected')
Map.addLayer(img.clip(geom), {'bands':['B4', 'B3','B2'], 'min':0, 'max':5000}, 'Mosaic IS Corrected')

In [34]:
CorCol = ee.ImageCollection(SrList)#.map(cloud_mask.sentinel2()) #converting the list of atcor images to an imagecollection
assetlocation = assetPath+assetID #concatenate string variables to make one save destination 

In [35]:
batch.ImageCollection.toAsset(col=CorCol, assetPath=assetlocation, scale=10, region=studyarea)#,create=True,verbose=False)

[<Task EXPORT_IMAGE: 0 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: 1 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: 2 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: 3 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: 4 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: 5 (UNSUBMITTED)>]

In [ ]:
# # export
export = ee.batch.Export.image.toAsset(\
    image=output,
    description='sentinel2_atmcorr_export',
    assetId = assetID,
    region = region,
    scale = 30)

batch.ImageColllection.toDrive(CorCol, property='site', folder='tools_exportbyfeat', name='test', scale=10, dataType='float')
ee.batch.export.image.toDrive(CorCol)
# # uncomment to run the export
#export.start() 